In [9]:
import torch
from collections import Counter


In [10]:
def unit_step_with_rand(x):
    return torch.where(x >= torch.rand(1), torch.tensor(1.0), torch.tensor(0.0))


def transform(tensor: torch.Tensor):
    return unit_step_with_rand(torch.sigmoid(tensor))

def number_from_tensor(tensor: torch.Tensor):
    powers_of_two = torch.vander(torch.Tensor([2]), N=tensor.size(dim=0)+1, increasing=True)
    scalar_mul = tensor * torch.reshape(powers_of_two, (-1,))[1:]
    return 1 + scalar_mul.sum()

In [11]:
class MyIntegerFactorizationModel:

    def __init__(self, F: int):
        self.F = F
        length = F.bit_length()
        self.P = (length - 1) // 2
        self.Q = length - 2 - self.P
        self.eternal_tensor = torch.ones(self.P + self.Q, requires_grad=True)
        self.tensor_collector = Counter()
        self.other_tensor_collector = Counter()

    def energy_function(self, tensor: torch.Tensor, i_0) -> torch.Tensor:
        return i_0 * (number_from_tensor(tensor[:self.P]) * number_from_tensor(tensor[self.P:]) - self.F) ** 2

    def evaluate(self, times: int):
        for i in range(times):
            first_number = number_from_tensor(self.eternal_tensor[:self.P])
            second_number = number_from_tensor(self.eternal_tensor[self.P:])
            self.tensor_collector[first_number.item()] += 1
            self.other_tensor_collector[second_number.item()] += 1
            energy = self.energy_function(self.eternal_tensor, 0.005)
            energy.backward(torch.ones(energy.shape))
            gradient = self.eternal_tensor.grad
            trans = transform(-gradient)
            with torch.no_grad():
                self.eternal_tensor.data = trans

In [12]:
integer_factorization = MyIntegerFactorizationModel(35)

integer_factorization.evaluate(40)
print(integer_factorization.tensor_collector)

Counter({1.0: 26, 7.0: 11, 3.0: 3})
